In [96]:
import requests as r
import pymysql as mysql
import datetime
import time




In [344]:


class league_match(object):
    
    def __init__(self,journey,homeTeam,homeTeam_id,awayTeam,awayTeam_id,date,goalsAwayTeam,goalsHomeTeam,league_id):
        self.journey=journey
        self.homeTeam=homeTeam
        self.awayTeam=awayTeam
        self.homeTeam_id=homeTeam_id
        self.awayTeam_id=awayTeam_id
        self.date=date
        self.goalsAwayTeam=goalsAwayTeam
        self.goalsHomeTeam=goalsHomeTeam
        self.league_id=league_id
        self.index=str(journey)+str(homeTeam_id)+str(awayTeam_id)+str(league_id)

class team(object):
    
    def __init__(self,name,id_,matchday,rank,playedGame,points,wins,losses):
        
        self.name=name
        self.id_=id_
        self.matchday=matchday
        self.rank=rank
        self.playedGame=playedGame
        self.points=points
        self.wins=wins
        self.losses=losses
        self.index=str(id_)+str(matchday)

class league(object):
    
    def __init__(self,league_id,api_key):
        self.api_key=api_key
        self.league_id=league_id
        self.teams={}
        self.matchs=[]
        self.get_list_team()
        self.get_matchs()
        
        
    def add_club_name(self,club_name,club_id):
        
        self.clubs[club_name]=club_id
        
        
    def add_match(self,match):
        
        self.matchs.append(match)
        
        
    def get_list_team(self):
        
        
        url="http://api.football-data.org/v1/soccerseasons/"+str(self.league_id)+"/leagueTable"
        
        headers = {"X-Auth-Token": self.api_key}
        
        data=r.get(url,headers=headers).json()
        
        matchday=data["matchday"]

        for team_ in data["standing"]:
    
            self.teams[team_["teamName"]]=team(team_["teamName"],team_["_links"]["team"]["href"][-3:],matchday,team_["position"],team_["playedGames"],team_["points"],team_["wins"],team_["losses"])
    

            
       
    def get_matchs(self):
        
        
        url='http://api.football-data.org/v1/soccerseasons/'+str(self.league_id)+'/fixtures'
        headers = {"X-Auth-Token": self.api_key}
        data=r.get(url,headers=headers)
        data=data.json()
        for d in data['fixtures']:
            
            if d["status"]=="FINISHED":
                league.add_match(self,league_match(journey=d["matchday"]
                                        ,homeTeam=d["homeTeamName"]
                                        ,awayTeam=d["awayTeamName"]
                                        ,homeTeam_id=d["_links"]["homeTeam"]["href"][-3:]
                                        ,awayTeam_id=d["_links"]["awayTeam"]["href"][-3:]     
                                        ,date=d["date"][:10]
                                        ,goalsAwayTeam=d["result"]["goalsAwayTeam"]
                                        ,goalsHomeTeam=d["result"]["goalsHomeTeam"]
                                        ,league_id=self.league_id))

class amz_rds(object):
    
    
    def __init__(self,host,username,password,db,port):
        
        
        self.host=host
        self.username=username
        self.password=password
        self.db=db
        self.port=port
        self.cnx=None
        
    def open_cnx(self,state):
         
        if state==True:
            
            self.cnx=mysql.Connect(self.host,self.username,self.password, self.db,self.port)
        
        elif state==False:
            
            self.cnx=mysql.Connect(self.host,self.username,self.password, self.db,self.port).close()
        else :
            
            raise ValueError('Input not understood')

            
    def add_match(self,cnx,journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_):
        
        cursor = cnx.cursor()
        
        add_match = ("INSERT INTO matchs_results "
                   "(journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_) "
                   "VALUES (%s, %s,%s, %s,%s, %s,%s)"
                    "ON DUPLICATE KEY UPDATE "
                    "journey=Values(journey),"
                    "homeTeam_id=VALUES(homeTeam_id),"
                    "awayTeam_id=VALUES(awayTeam_id),"
                    "goalsHomeTeam=VALUES(goalsHomeTeam),"
                    "goalsAwayTeam=VALUES(goalsAwayTeam),"
                    "date=VALUES(date),"
                    "index_=Values(index_);")   

        data_match = (journey, homeTeam_id,awayTeam_id,goalsHomeTeam,goalsAwayTeam,date,index_)

        cursor.execute(add_match, data_match)

        emp_no = cursor.lastrowid

        cnx.commit()

        cursor.close()
        
    def add_all_match(self,league):
        
        for m in league.matchs:
            
            self.add_match(self.cnx,m.journey,m.homeTeam_id,m.awayTeam_id,m.goalsHomeTeam,m.goalsAwayTeam,m.date,m.index)
    
    def add_team(self,cnx,name,id_,matchday,rank,points,wins,losses):
        
        cursor = cnx.cursor()
        index=str((id_))+str(matchday)
        add_team = ("INSERT INTO teams_ranking "
                   "(name,id_,matchday,rank,points,wins,losses,index_) "
                   "VALUES (%s, %s,%s, %s,%s, %s,%s,%s)"
                    "ON DUPLICATE KEY UPDATE "
                    "name=Values(name),"
                    "id_=VALUES(id_),"
                    "matchday=VALUES(matchday),"
                    "rank=VALUES(rank),"
                    "points=VALUES(points),"
                    "wins=Values(wins),"
                    "losses=Values(losses);")   

        data_team = (name,id_,matchday,rank,points,wins,losses,index)

        cursor.execute(add_team, data_team)

        emp_no = cursor.lastrowid

        cnx.commit()

        cursor.close()

            
    def add_all_teams(self,league):    
        
        for t in league.teams.values():
            
            
            
            self.add_team(self.cnx,t.name,t.id_,t.matchday,t.rank,t.points,t.wins,t.losses)
        